In [16]:
import os
import io
import ast
import boto3
import sagemaker
import numpy as np
import pandas as pd

from sagemaker.pytorch import PyTorch

from ibov.utils import load_config
from ibov.deploy import get_deploy_config, define_model

ImportError: cannot import name 'define_model' from 'ibov.deploy' (/home/aadelmo/projetos/udacity/BovespaStockPrediction/ibov/deploy.py)

### Loading Configs

In [21]:
# Load config dict
config = load_config()

In [22]:
role = config.get("sagemaker").get("role")
prefix = config.get("sagemaker").get("bucket_prefx")
data_dir = config.get("data").get("dir")
region = config.get("sagemaker").get("region")

In [23]:
dropout = config.get("model").get("dropout")
window = config.get("feature").get("window")
hidden_layer = config.get("model").get("hidden_layer")
lr = config.get("model").get("lr")
seed = config.get("model").get("seed")
epochs = config.get("model").get("epochs")

### Upload Data

In [5]:
session = sagemaker.Session(boto_session=boto3.session.Session(region_name=region))
bucket = session.default_bucket()

In [6]:
input_data = session.upload_data(path=data_dir, bucket=bucket, key_prefix=prefix)
input_config = session.upload_data(path="config.json", bucket=bucket, key_prefix=prefix)

### Train Model

In [7]:
estimator = PyTorch(entry_point="train.py", 
                    source_dir="ibov", 
                    py_version="py3",
                    role=role, 
                    framework_version='0.4.0',
                    instance_count=1, 
                    instance_type='ml.p2.xlarge',
                    hyperparameters = {
                        "batch-size": 50,
                        "epochs": epochs,
                        "seed": seed,
                        "input-layer": window,
                        "hidden-layer": hidden_layer,
                        "dropout": dropout        
                    })

In [8]:
estimator.fit({'train': input_data, "config": input_config})

2021-01-25 22:23:41 Starting - Starting the training job...
2021-01-25 22:24:05 Starting - Launching requested ML instancesProfilerReport-1611613418: InProgress
......
2021-01-25 22:25:16 Starting - Preparing the instances for training.........
2021-01-25 22:27:07 Downloading - Downloading input data
2021-01-25 22:27:07 Training - Downloading the training image..bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2021-01-25 22:27:24,345 sagemaker-containers INFO     Imported framework sagemaker_pytorch_container.training
2021-01-25 22:27:24,386 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2021-01-25 22:27:27,427 sagemaker_pytorch_container.training INFO     Invoking user training script.
2021-01-25 22:27:27,708 sagemaker-containers INFO     Module train does not provide a setup.py. 
Generating setup.py
2021-01-25 22:27:27,708 sagemaker-containers INFO     Generating setup.cfg
2021-01-

22:27:50, epoch: 9, train: 0.54, valid: 0.042
22:27:53, epoch: 10, train: 0.571, valid: 0.04
22:27:56, epoch: 11, train: 0.561, valid: 0.035
22:28:00, epoch: 12, train: 0.545, valid: 0.026
22:28:04, epoch: 13, train: 0.558, valid: 0.026
22:28:09, epoch: 14, train: 0.485, valid: 0.035
22:28:14, epoch: 15, train: 0.495, valid: 0.028
22:28:18, epoch: 16, train: 0.486, valid: 0.028
22:28:24, epoch: 17, train: 0.52, valid: 0.03
22:28:29, epoch: 18, train: 0.507, valid: 0.027
22:28:35, epoch: 19, train: 0.502, valid: 0.033
22:28:41, epoch: 20, train: 0.497, valid: 0.04
22:28:47, epoch: 21, train: 0.482, valid: 0.043
22:28:54, epoch: 22, train: 0.469, valid: 0.027
22:29:00, epoch: 23, train: 0.438, valid: 0.027
22:29:09, epoch: 24, train: 0.433, valid: 0.032
22:29:16, epoch: 25, train: 0.454, valid: 0.037
22:29:24, epoch: 26, train: 0.45, valid: 0.031
22:29:32, epoch: 27, train: 0.403, valid: 0.039
22:29:40, epoch: 28, train: 0.425, valid: 0.033
22:29:49, epoch: 29, train: 0.425, valid: 0.027

In [25]:
estimator.latest_training_job.job_name

'sagemaker-pytorch-2021-01-25-22-23-37-005'

In [26]:
deploy_config = get_deploy_config(config)

In [27]:
def define_model(deploy_config):

    codename = deploy_config.get("codename")
    
    client  = boto3.client("sagemaker")

    model = client.create_model(ModelName=codename,
                                PrimaryContainer=deploy_config.get("primary_container"),
                                ExecutionRoleArn=deploy_config.get("role"))

    return model

In [28]:
define_model(deploy_config)

{'ModelArn': 'arn:aws:sagemaker:us-east-1:977053370764:model/sagemaker-pytorch-2021-01-25-22-23-37-005',
 'ResponseMetadata': {'RequestId': 'cdf8cd34-7144-48cf-b4a9-fc01db859e64',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'cdf8cd34-7144-48cf-b4a9-fc01db859e64',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '103',
   'date': 'Tue, 26 Jan 2021 00:30:29 GMT'},
  'RetryAttempts': 0}}

### Endpoint Deployment

config = load_config()

deploy_config = get_deploy_config(config)

build_endpoint(deploy_config)

kill_endpoint(config)